In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [2]:
source('../parameters.sh')
source('parameters.sh')



In [3]:
file.path(data_d, str_replace(RData_path, '.RData', ''), variant_weights_f) %>%
fread(colClasses = c('#CHROM'='character', 'POS_total' = 'numeric')) %>%
rename('CHROM'='#CHROM') -> df


In [12]:
df %>% 
mutate(rank_beta = rank(-abs(HC382)), POS_e = POS + str_length(REF)) %>%
filter(rank_beta <= 5000) %>%
select(CHROM, POS, POS_e, ID, HC382, rank_beta) %>%
arrange(rank_beta) %>% head()

CHROM,POS,POS_e,ID,HC382,rank_beta
<chr>,<int>,<int>,<chr>,<dbl>,<dbl>
5,131901225,131901226,rs2244012,0.02686650,1
17,4535314,4535315,rs34210653,-0.02642665,2
9,6193455,6193456,rs2381416,-0.02515378,3
5,110401872,110401873,rs1837253,0.02483261,4
15,67442596,67442597,rs17293632,0.02451457,5
1,152179152,152179153,rs12123821,0.02281375,6


In [10]:
plot_cols <- c(
    'HC382',
    'INI30130', 'INI30140', 'INI30150', 'INI30160', 'INI3062', 'INI3064', 'INI3063',
    paste0('Component', 1:8)
)


In [19]:
great_ucsc_bed_f <- file.path(data_d, str_replace(RData_path, '.RData', ''), 'GREAT_in_bed', paste0(plot_col, '.bed'))

In [20]:
great_ucsc_bed_f

[1] "/oak/stanford/groups/mrivas/projects/biobank-methods-dev/20210422_SRRR/Asthma/results_rank_5/output_lambda_67/GREAT_in_bed/HC382.bed"

In [25]:
plot_col <- plot_cols[1]


df %>%
rename(!!'plotCol' := all_of(plot_col)) %>%
mutate(rank_beta = rank(-abs(plotCol)), POS_e = POS + str_length(REF)) %>%
filter(rank_beta <= 5000) %>%
mutate(
    CHROM = if_else(CHROM == 'XY', 'X', CHROM),
    CHROM = if_else(CHROM == 'MT', 'M', CHROM),
    CHROM = paste0('chr', CHROM)
) %>%
select(CHROM, POS, POS_e, ID) %>%
fwrite(great_ucsc_bed_f, sep='\t', na = "NA", quote=F, col.names=F)


In [26]:
for(plot_col in plot_cols){
    great_ucsc_bed_f <- file.path(data_d, str_replace(RData_path, '.RData', ''), 'GREAT_in_bed', paste0(plot_col, '.bed'))
    
    df %>%
    rename(!!'plotCol' := all_of(plot_col)) %>%
    mutate(rank_beta = rank(-abs(plotCol)), POS_e = POS + str_length(REF)) %>%
    filter(rank_beta <= 5000) %>%
    mutate(
        CHROM = if_else(CHROM == 'XY', 'X', CHROM),
        CHROM = if_else(CHROM == 'MT', 'M', CHROM),
        CHROM = paste0('chr', CHROM)
    ) %>%
    select(CHROM, POS, POS_e, ID) %>%
    fwrite(great_ucsc_bed_f, sep='\t', na = "NA", quote=F, col.names=F)
}


